In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from mcp.types import Tool, ToolAnnotations
import os 
import wandb
import torch
import json
from transformers import DataCollatorForSeq2Seq
from unsloth.chat_templates import train_on_responses_only


In [ ]:
os.environ['WANDB_API_KEY'] = "XXXX"
os.environ['WANDB_PROJECT'] = "qwen3"

wandb.login()

In [ ]:
max_seq_length = 8128 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = False,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)


model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-3",
)

In [ ]:
dataset = load_dataset("jdaddyalbs/playwright-mcp-toolcalling", split="train")

In [ ]:
tools = [Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, description='Returns all console messages', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Get console messages', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_handle_dialog', title=None, description='Handle a dialog', inputSchema={'type': 'object', 'properties': {'accept': {'type': 'boolean', 'description': 'Whether to accept the dialog.'}, 'promptText': {'type': 'string', 'description': 'The text of the prompt in case of a prompt dialog.'}}, 'required': ['accept'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Handle a dialog', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_file_upload', title=None, description='Upload one or multiple files', inputSchema={'type': 'object', 'properties': {'paths': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The absolute paths to the files to upload. Can be a single file or multiple files.'}}, 'required': ['paths'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Upload files', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_install', title=None, description='Install the browser specified in the config. Call this if you get an error about the browser not being installed.', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Install the browser specified in the config', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_press_key', title=None, description='Press a key on the keyboard', inputSchema={'type': 'object', 'properties': {'key': {'type': 'string', 'description': 'Name of the key to press or a character to generate, such as `ArrowLeft` or `a`'}}, 'required': ['key'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Press a key', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_navigate', title=None, description='Navigate to a URL', inputSchema={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL to navigate to'}}, 'required': ['url'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Navigate to a URL', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_navigate_back', title=None, description='Go back to the previous page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Go back', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_navigate_forward', title=None, description='Go forward to the next page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Go forward', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_network_requests', title=None, description='Returns all network requests since loading the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='List network requests', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_pdf_save', title=None, description='Save page as PDF', inputSchema={'type': 'object', 'properties': {'filename': {'type': 'string', 'description': 'File name to save the pdf to. Defaults to `page-{timestamp}.pdf` if not specified.'}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Save as PDF', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_take_screenshot', title=None, description="Take a screenshot of the current page. You can't perform actions based on the screenshot, use browser_snapshot for actions.", inputSchema={'type': 'object', 'properties': {'raw': {'type': 'boolean', 'description': 'Whether to return without compression (in PNG format). Default is false, which returns a JPEG image.'}, 'filename': {'type': 'string', 'description': 'File name to save the screenshot to. Defaults to `page-{timestamp}.{png|jpeg}` if not specified.'}, 'element': {'type': 'string', 'description': 'Human-readable element description used to obtain permission to screenshot the element. If not provided, the screenshot will be taken of viewport. If element is provided, ref must be provided too.'}, 'ref': {'type': 'string', 'description': 'Exact target element reference from the page snapshot. If not provided, the screenshot will be taken of viewport. If ref is provided, element must be provided too.'}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Take a screenshot', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_snapshot', title=None, description='Capture accessibility snapshot of the current page, this is better than screenshot', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Page snapshot', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_click', title=None, description='Perform click on a web page', inputSchema={'type': 'object', 'properties': {'element': {'type': 'string', 'description': 'Human-readable element description used to obtain permission to interact with the element'}, 'ref': {'type': 'string', 'description': 'Exact target element reference from the page snapshot'}, 'doubleClick': {'type': 'boolean', 'description': 'Whether to perform a double click instead of a single click'}}, 'required': ['element', 'ref'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Click', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_drag', title=None, description='Perform drag and drop between two elements', inputSchema={'type': 'object', 'properties': {'startElement': {'type': 'string', 'description': 'Human-readable source element description used to obtain the permission to interact with the element'}, 'startRef': {'type': 'string', 'description': 'Exact source element reference from the page snapshot'}, 'endElement': {'type': 'string', 'description': 'Human-readable target element description used to obtain the permission to interact with the element'}, 'endRef': {'type': 'string', 'description': 'Exact target element reference from the page snapshot'}}, 'required': ['startElement', 'startRef', 'endElement', 'endRef'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Drag mouse', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_hover', title=None, description='Hover over element on page', inputSchema={'type': 'object', 'properties': {'element': {'type': 'string', 'description': 'Human-readable element description used to obtain permission to interact with the element'}, 'ref': {'type': 'string', 'description': 'Exact target element reference from the page snapshot'}}, 'required': ['element', 'ref'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Hover mouse', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_type', title=None, description='Type text into editable element', inputSchema={'type': 'object', 'properties': {'element': {'type': 'string', 'description': 'Human-readable element description used to obtain permission to interact with the element'}, 'ref': {'type': 'string', 'description': 'Exact target element reference from the page snapshot'}, 'text': {'type': 'string', 'description': 'Text to type into the element'}, 'submit': {'type': 'boolean', 'description': 'Whether to submit entered text (press Enter after)'}, 'slowly': {'type': 'boolean', 'description': 'Whether to type one character at a time. Useful for triggering key handlers in the page. By default entire text is filled in at once.'}}, 'required': ['element', 'ref', 'text'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Type text', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_select_option', title=None, description='Select an option in a dropdown', inputSchema={'type': 'object', 'properties': {'element': {'type': 'string', 'description': 'Human-readable element description used to obtain permission to interact with the element'}, 'ref': {'type': 'string', 'description': 'Exact target element reference from the page snapshot'}, 'values': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Array of values to select in the dropdown. This can be a single value or multiple values.'}}, 'required': ['element', 'ref', 'values'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Select option', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_tab_list', title=None, description='List browser tabs', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='List tabs', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_tab_new', title=None, description='Open a new tab', inputSchema={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL to navigate to in the new tab. If not provided, the new tab will be blank.'}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Open a new tab', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_tab_select', title=None, description='Select a tab by index', inputSchema={'type': 'object', 'properties': {'index': {'type': 'number', 'description': 'The index of the tab to select'}}, 'required': ['index'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Select a tab', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_tab_close', title=None, description='Close a tab', inputSchema={'type': 'object', 'properties': {'index': {'type': 'number', 'description': 'The index of the tab to close. Closes current tab if not provided.'}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close a tab', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_generate_playwright_test', title=None, description='Generate a Playwright test for given scenario', inputSchema={'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the test'}, 'description': {'type': 'string', 'description': 'The description of the test'}, 'steps': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The steps of the test'}}, 'required': ['name', 'description', 'steps'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Generate a Playwright test', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_wait_for', title=None, description='Wait for text to appear or disappear or a specified time to pass', inputSchema={'type': 'object', 'properties': {'time': {'type': 'number', 'description': 'The time to wait in seconds'}, 'text': {'type': 'string', 'description': 'The text to wait for'}, 'textGone': {'type': 'string', 'description': 'The text to wait for to disappear'}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Wait for', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None)]

In [ ]:
tools_json = [
    {
        "type":"function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema
            #"parameters": json.dumps(tool.inputSchema)
        }
    } for tool in tools
]

In [ ]:
dataset = dataset.map(lambda x: {"conversations":x['messages'][:3]})

In [ ]:
def apply_template(examples):
    messages = examples["conversations"]
    text = [
        tokenizer.apply_chat_template(
            message,
            tools=tools_json,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        ) 
        for message in messages
    ]
    return {"text": text}

In [ ]:
dataset = dataset.map(apply_template, batched=True)
ds = dataset.train_test_split(test_size = 0.1)
train_dataset = ds['train']
eval_dataset = ds['test']

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset, # Can set up evaluation!
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1, # could probably do 128
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        output_dir='qwen3-sft',
        dataset_num_proc=2,
        eval_steps=50,
        fp16_full_eval = True,
        per_device_eval_batch_size = 1,
        eval_accumulation_steps = 1,
        eval_strategy = "steps",
    ),
)

In [ ]:
train_dataset['text'][0]

In [ ]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint=False)